In [27]:
#r "nuget: Azure.Identity, 1.13.2"
#r "nuget: Microsoft.SemanticKernel, 1.41.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.41.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.41.0-preview"
#r "nuget: Microsoft.SemanticKernel.Agents.AzureAI, 1.41.0-preview"
#r "nuget: Microsoft.SemanticKernel.Prompty, 1.41.0-alpha"
using Azure.Identity;
using Azure.AI.Projects;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.AzureAI;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Threading;
using Kernel = Microsoft.SemanticKernel.Kernel;
#!import tools/Settings.cs
#!import tools/Extensions.cs

Installed Packages Azure.Identity, 1.13.2 Microsoft.SemanticKernel, 1.41.0 Microsoft.SemanticKernel.Agents.AzureAI, 1.41.0-preview Microsoft.SemanticKernel.Agents.Core, 1.41.0-preview Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.41.0 Microsoft.SemanticKernel.Prompty, 1.41.0-alpha

In [2]:
var projectClient = new AIProjectClient(Settings.AIFoundryProjectConnectionString, new DefaultAzureCredential());
var agentsClient = projectClient.GetAgentsClient();

### Create the agents in Azure AI and then the corresponding Semantic Kernel agent objects

In [28]:
#pragma warning disable SKEXP0110

var copyWriterAgentAzDefinition = await agentsClient.CreateOrUpdateAgentAsync(
    model: Settings.ModelDeploymentName,
    name: "CopyWriter",
    description: "copywriter known for brevity and a dry humor",
    instructions:
        """
        You are a copywriter with ten years of experience and are known for brevity and a dry humor.
        The goal is to refine and decide on the single best copy as an expert in the field.
        Only provide a single proposal per response.
        You're laser focused on the goal at hand.
        Don't waste time with chit chat.
        Consider suggestions when refining an idea.
        """
);
var copyWriterAgent = new AzureAIAgent(copyWriterAgentAzDefinition, agentsClient);

var artDirectorAgentAzDefinition = await agentsClient.CreateOrUpdateAgentAsync(
    model: Settings.ModelDeploymentName,
    name: "ArtDirector",
    description: "art director with opinions about copywriting born of a love for David Ogilvy",
    instructions:
        """
        You are an art director who has opinions about copywriting born of a love for David Ogilvy.
        The goal is to determine if the given copy is acceptable to print.
        If so, state that it is approved.  Do not use the word "approve" unless you are giving approval.
        If not, provide insight on how to refine suggested copy without example.
        """
);
var artDirectorAgent = new AzureAIAgent(artDirectorAgentAzDefinition, agentsClient);

Agent with name 'CopyWriter' already exists. Updating the existing agent.
Agent with name 'ArtDirector' already exists. Updating the existing agent.


### Create the Agent Group Chat object.

This sets up an `AgentGroupChat` with two agents - since there is no selection strategy, the agents will sequentially execute indefinitely until:
* Limit of 10 turns is reached
* Custom Termination strategy detects that the `"ArtDirector"` agent used the phrase `"approve"`

In [36]:
#pragma warning disable SKEXP0110

var groupChat = new AgentGroupChat(copyWriterAgent, artDirectorAgent)
{
    ExecutionSettings = new(){
        // Terminate when the Art Directors's message contains the term "approve"
        TerminationStrategy = new RegexTerminationStrategy("approve") {
            MaximumIterations = 10,
            Agents = [artDirectorAgent]
        }
    }
};


### Execute the Group chat using a staring prompt

In [38]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

var input = new ChatMessageContent(AuthorRole.User, "concept: maps made out of egg cartons.");

try
{
    groupChat.AddChatMessage(input);

    string lastCopyWriterOutput = null;

    await foreach (var response in groupChat.InvokeAsync())
    {
        Console.WriteLine($"[{response.Role} - {response.AuthorName}] {response.Content}");

        if (response.Role == AuthorRole.Assistant && response.AuthorName == copyWriterAgent.Name)
        {
            lastCopyWriterOutput = response.Content;
        }
    }
    
    Console.WriteLine($"\n[IS COMPLETED: {groupChat.IsComplete}]");

    if (lastCopyWriterOutput is null)
    {
        Console.WriteLine("[TERMINATED WITHOUT COPYWRITER OUTPUT]");
    }
    else
    {
        Console.WriteLine($"[FINAL COPYWRITER OUTPUT] {lastCopyWriterOutput}");
    }
}
finally
{
    await groupChat.ResetAsync();
}

[assistant - CopyWriter] "Navigate the world, one carton at a time."
[assistant - ArtDirector] This copy has potential, but it feels lackluster and doesn't evoke any intrigue or differentiation. It needs to highlight the creativity, sustainability, or artistry behind using egg cartons as maps. Focus on bringing emotional resonance or practical value to the fore. Suggest an angle that makes it more memorable and distinct.
[assistant - CopyWriter] "Chart your world—recycled and redesigned."
[assistant - ArtDirector] This copy leans toward being concise and slightly evocative, but it misses the mark on capturing the unique, tactile, and creative nature of egg carton maps. It needs to evoke a vivid image or curiosity in the audience. Explore a more imaginative or sensory-focused approach that embodies the handmade artistry.
[assistant - CopyWriter] "Your map, cracked yet crafted."
[assistant - ArtDirector] This feels overly ambiguous and doesn’t effectively convey the ingenuity or visual a